# Projeto Final de Spark

*Diego Silva Cunha*

**Nível Básico:**

Dados: https://mobileapps.saude.gov.br/esus-vepi/files/unAFkcaNDeXajurGB7LChj8SgQYS2ptm/04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar

Referência das Visualizações:
* Site: https://covid.saude.gov.br/
* Guia do Site: Painel Geral

**1. Enviar os dados para o hdfs**

Salvei os arquivos dentro da pasta /input/dados e faço o envio dos arquivos para o HDFS onde será mapeada a tabela Hive 'painel_covid_temp'.
```
hdfs dfs -put /input/dados /user/diego/trabalho-final/data/painel_covid_temp
```

**2. Otimizar todos os dados do hdfs para uma tabela Hive particionada por município.**

Conectar ao Hive com o comando abaixo:
```
beeline -u jdbc:hive2://localhost:10000
```
Criando o banco de dados Hive
```
create database trabalhofinal;
use trabalhofinal;
```
Criando tabela Hive 'painel_covid_temp'
```
create external table painel_covid_temp (
    regiao string,
    estado string,
    municipio string,
    coduf int,
    codmun int,
    codRegiaoSaude int,
    nomeRegiaoSaude string,
    data date,
    semanaEpi int,
    populacaoTCU2019 int,
    casosAcumulado int,
    casosNovos int,
    obitosAcumulado int,
    obitosNovos int,
    Recuperadosnovos int,
    emAcompanhamentoNovos int,
    interior_metropolitana int
)
row format delimited
fields terminated by ';'
location'/user/diego/trabalho-final/data/painel_covid_temp'
tblproperties("skip.header.line.count"="1");
```
Realizando a configuração do Hive para suportar particionamento dinâmico.
```
SET hive.exec.dynamic.partition = true;
SET hive.exec.dynamic.partition.mode = nonstrict;
```
Criando a tabela 'painel_covid' particionada por regiões do Brasil.
```
create table painel_covid (
    estado string,
    municipio string,
    coduf int,
    codmun int,
    codRegiaoSaude int,
    nomeRegiaoSaude string,
    data date,
    semanaEpi int,
    populacaoTCU2019 int,
    casosAcumulado int,
    casosNovos int,
    obitosAcumulado int,
    obitosNovos int,
    Recuperadosnovos int,
    emAcompanhamentoNovos int,
    interior_metropolitana int
)
partitioned by (regiao string);
```
Carregando os dados da tabela 'painel_covid_temp' para a tabela particionada por região 'painel_covid'.
```
insert overwrite table painel_covid partition(regiao) select estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior_metropolitana,regiao from painel_covid_temp;
```

In [31]:
from pyspark.sql.functions import *
from pyspark.sql.types import FloatType

def calcular_incidencia(casos, populacao, por_habitantes):
    return format_number((casos * por_habitantes) / populacao, 2)

def calcular_mortalidade(obitos, populacao, por_habitantes):
    return format_number((obitos * por_habitantes) / populacao, 2)

def calcular_letalidade(obitos, casos):
    return format_number((obitos * 100) / casos, 2)

spark.sparkContext.setLogLevel("INFO")
spark.catalog.setCurrentDatabase(dbName="trabalhofinal")

**3. Criar as 3 vizualizações pelo Spark com os dados enviados para o HDFS:**

In [2]:
painel_covid = spark.read.table("painel_covid")

Vizualização 1 - Casos recuperados e em acompahamento

In [9]:
vizualizacao_1 = painel_covid.select("recuperadosnovos","emacompanhamentonovos")\
            .agg(max("recuperadosnovos").alias("recuperadosnovos"),\
                 max("emacompanhamentonovos").alias("emacompanhamentonovos"))
vizualizacao_1.show()

+----------------+---------------------+
|recuperadosnovos|emacompanhamentonovos|
+----------------+---------------------+
|        17262646|              1317658|
+----------------+---------------------+



Vizualização 2 - Casos confirmados

In [32]:
vizualizacao_2 = painel_covid.select("casosacumulado","casosnovos","populacaoTCU2019")\
            .agg(max("casosacumulado").alias("casosacumulado"),\
                 max("casosnovos").alias("casosnovos"),\
                 max("populacaoTCU2019").alias("populacaoTCU2019"))\
            .withColumn("incidencia*",
                        calcular_incidencia(col("casosacumulado").cast(FloatType()),
                                            col("populacaoTCU2019").cast(FloatType()),
                                            1000000))
vizualizacao_2.show()

+--------------+----------+----------------+-----------+
|casosacumulado|casosnovos|populacaoTCU2019|incidencia*|
+--------------+----------+----------------+-----------+
|      18855015|    115228|       210147125|  89,722.93|
+--------------+----------+----------------+-----------+



Vizualização 3 - Óbitos confirmados

In [33]:
vizualizacao_3 = painel_covid.select("casosacumulado","obitosacumulado","obitosnovos","populacaoTCU2019")\
            .agg(max("casosacumulado").alias("casosacumulado"),
                 max("obitosacumulado").alias("obitosacumulado"),
                 max("obitosnovos").alias("obitosnovos"),
                 max("populacaoTCU2019").alias("populacaoTCU2019"))\
            .withColumn("letalidade", 
                        calcular_letalidade(col("obitosacumulado").cast(FloatType()),
                                            col("casosacumulado").cast(FloatType())))\
            .withColumn("mortalidade", 
                        calcular_mortalidade(col("obitosacumulado").cast(FloatType()),
                                             col("populacaoTCU2019").cast(FloatType()),1000000))
vizualizacao_3.show()

+--------------+---------------+-----------+----------------+----------+-----------+
|casosacumulado|obitosacumulado|obitosnovos|populacaoTCU2019|letalidade|mortalidade|
+--------------+---------------+-----------+----------------+----------+-----------+
|      18855015|         526892|       4249|       210147125|      2.79|   2,507.25|
+--------------+---------------+-----------+----------------+----------+-----------+



**4. Salvar a primeira visualização como tabela Hive**

In [210]:
vizualizacao_1.write.saveAsTable("trabalhofinal.v1_casos_recuperados")

**5. Salvar a segunda visualização com formato parquet e compressão snappy**

In [215]:
vizualizacao_2.write.option("compression", "snappy")\
                    .parquet("/user/diego/trabalho-final/data/v2_casos_confirmados")

**6. Salvar a terceira visualização em um tópico no Kafka**

In [46]:
vizualizacao_3.selectExpr("CONCAT(1) AS value", "CAST(obitosacumulado AS STRING)","CAST(obitosnovos AS STRING)","CAST(letalidade AS STRING)","CAST(mortalidade AS STRING)")\
            .write\
            .format("kafka")\
            .option("kafka.bootstrap.servers","kafka:9092")\
            .option("topic","v3-obitos-confirmados")\
            .option("value", "1")\
            .option("checkpointLocation","/user/diego/trabalho-final/data/kafka/checkpoint")\
            .save()

**7. Criar a visualização pelo Spark com os dados enviados para o HDFS:**

In [34]:
vizualizacao_regioes = painel_covid.where((col("municipio")!="") | (col("regiao")=="Brasil"))\
            .groupBy("regiao","municipio")\
            .agg(max("casosacumulado").alias("casosacumulado"),
                 max("obitosacumulado").alias("obitosacumulado"),
                 max("populacaoTCU2019").alias("populacaoTCU2019"))\
            .groupBy("regiao")\
            .agg(sum("casosacumulado").alias("casosacumulado"),
                 sum("obitosacumulado").alias("obitosacumulado"),
                 max("populacaoTCU2019").alias("populacaoTCU2019"))\
            .withColumn("incidencia100mil",
                        calcular_incidencia(col("casosacumulado").cast(FloatType()),
                                            col("populacaoTCU2019").cast(FloatType()),
                                            100000))\
            .withColumn("mortalidade100mil",
                        calcular_mortalidade(col("obitosacumulado").cast(FloatType()),
                                             col("populacaoTCU2019").cast(FloatType()),
                                             100000))
vizualizacao_regioes.show()

+------------+--------------+---------------+----------------+----------------+-----------------+
|      regiao|casosacumulado|obitosacumulado|populacaoTCU2019|incidencia100mil|mortalidade100mil|
+------------+--------------+---------------+----------------+----------------+-----------------+
|    Nordeste|       4426773|         115502|         2872347|      154,116.93|         4,021.17|
|         Sul|       3780833|          85038|         1933105|      195,583.43|         4,399.04|
|     Sudeste|       7129027|         244856|        12252023|       58,186.53|         1,998.49|
|Centro-Oeste|       1916634|          49490|         3015268|       63,564.30|         1,641.31|
|      Brasil|      18855015|         526892|       210147125|        8,972.29|           250.73|
|       Norte|       1730152|          43929|         2182763|       79,264.31|         2,012.54|
+------------+--------------+---------------+----------------+----------------+-----------------+



**8. Salvar a visualização do exercício 6 em um tópico no Elastic**

**9. Criar um dashboard no Elastic para visualização dos novos dados enviados**